In [1]:
import os, sys
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 --jars /usr/local/jar_files/spark-streaming-kafka-0-8-assembly_2.11-2.3.2.jar pyspark-shell'

In [2]:
# Import findspark 
import findspark

# Initialize and provide path
findspark.init("/usr/share/spark/spark-2.3.2-bin-hadoop2.7/")

# Or use this alternative
findspark.init()

In [3]:
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    json parsing
import json
from pyspark.streaming.kafka import KafkaUtils

In [4]:
# Import SparkSession for opthist_coll
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Linear_Regression_Model") \
    .config("spark.mongodb.input.uri", "mongodb://localhost/stocks_db.opthist_coll") \
    .config("spark.mongodb.output.uri", "mongodb://localhost/stocks_db.opthist_coll") \
    .getOrCreate()
    
sc = spark.sparkContext

In [5]:
# Import SparkSession for stockhist_coll
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Linear_Regression_Model") \
    .config("spark.mongodb.input.uri", "mongodb://localhost/stocks_db.stockhist_coll") \
    .config("spark.mongodb.output.uri", "mongodb://localhost/stocks_db.stockhist_coll") \
    .getOrCreate()
    
sc = spark.sparkContext

In [6]:
#    Spark SQL
from pyspark.sql import Row,SQLContext

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

In [7]:
##   Definging write_mongo function, which loads df into database
import pymongo

def write_mongo(path):
    try:
        # add the input df to the database
        df = spark.read.json(path)
        df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()
        
    except:
        e = sys.exc_info()[0]
        print("Mongo Error: %s" % e)

In [8]:
# defining path_stock as path of previously downloaded historical stock data (reference: OLS_model_final)
path_stock = "./data1.txt"

In [9]:
# write the df to databse with pymongo and MongoDB SparkConnector
write_mongo(path_stock)       